In [ ]:
from datetime import datetime

def timespan(trange):
    # trangeを適切なフォーマットでパースして開始日と終了日を取得
    start_date = datetime.strptime(trange, '%Y-%m-%d')
    end_date = start_date  # 終了日は開始日として設定（仮の例）

    # データの読み込みや処理を行うために、start_dateとend_dateを使用する

    # 例: 開始日と終了日の範囲でデータを読み込む
    data = load_data(start_date, end_date)

    # 他の処理を実行する

# 使用例
trange = '2017-04-01'
timespan(trange)


In [ ]:
import urllib.request
import os
import numpy as np


def root_data_dir():
    return 'C:\data'  # データのルートディレクトリのパスを返す

import datetime

def file_dailynames(file_format, trange=None, times=None):
    if trange is None and times is None:
        raise ValueError("Either trange or times must be provided.")
    
    if trange is not None:
        start_date = trange[0]
        end_date = trange[1]
        num_days = (end_date - start_date).days + 1
        dates = [start_date + datetime.timedelta(days=i) for i in range(num_days)]
    else:
        dates = [dt.date() for dt in times]
    
    file_names = []
    for date in dates:
        file_name = date.strftime(file_format.replace('YYYY', '%Y').replace('MM', '%m').replace('DD', '%d'))
        file_names.append(file_name)
    
    return file_names

import numpy as np

def read_ascii(file_path, template, count):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    # データを格納するためのリストやNumPy配列を初期化
    data_list = []
    
    for line in lines:
        # 行のデータを処理して適切な形式に変換し、data_listに追加する
        # 例: スペース区切りの数値を格納する場合
        values = line.strip().split(' ')
        data = [float(value) for value in values]
        data_list.append(data)
    
    # データをIDLの変数に格納するための処理
    # 例: templateに応じてIDLの変数に代入する
    #     countに基づいてデータをフィルタリングするなど
    #     必要な処理に従ってデータを加工して返す
    
    # データの加工や変換が必要な場合は、ここで適切な処理を実装する
    
    return data_list  # 必要なデータを返す（IDLの変数に相当）




def load_fufp_txt(trange=None, err=None, file_src=None, high=None):
    err = 0
    
    if trange is not None:
        timespan(trange)
    
    template_file = r'/home/jackkooki/Document/duct_analysis/basic/erg_data/satellite/erg/pwe/hfa/event/hfa_l3_ne_temp_v03.sav'
    #restore, filename=template_file
    
    if high:
        remotedir = 'http://adrastea.gp.tohoku.ac.jp/~erg/data/hfa_l3_h/'
    else:
        remotedir = 'http://adrastea.gp.tohoku.ac.jp/~erg/data/hfa_l3/'
    
    localdir = os.path.join(root_data_dir(), 'ERG', 'HFA', 'L3', 'text')
    relfpathfmt = 'YYYY/hfa_l3_YYYYMMDD_v??_??.txt'
    relfpaths = file_dailynames(file_format=relfpathfmt, trange=trange, times=times)
    
    datfiles = []
    for relfpath in relfpaths:
        url = os.path.join(remotedir, relfpath)
        local_path = os.path.join(localdir, os.path.basename(relfpath))
        urllib.request.urlretrieve(url, local_path)
        datfiles.append(local_path)
    
    nd = len(datfiles)
    
    ts = []
    r = []
    glat = []
    mlt = []
    l_val = []
    b = []
    fce = []
    fuh = []
    fpe = []
    dne = []
    
    file_src = os.path.basename(datfiles[nd//2])
    for k in range(nd):
        hfa_l3 = read_ascii(datfiles[k], template=hfa_l3_ne_temp, count=count)
        if count == 0:
            continue
        
        n = len(hfa_l3.date)
        
        for i in range(n):
            yr = hfa_l3.date[i][:4]
            mn = hfa_l3.date[i][4:6]
            dy = hfa_l3.date[i][6:8]
            hr = int(hfa_l3.hour[i])
            mm = int(hfa_l3.hour[i]*60) - hr*60
            sc = int(hfa_l3.hour[i]*3600) - hr*3600 - mm*60
            ts.append(f'{yr}-{mn}-{dy}/{hr:02d}:{mm:02d}:{sc:02d}')
        
        r.append(hfa_l3.r)
        glat.append(hfa_l3.glat)
        mlt.append(hfa_l3.mlt)
        l_val.append(hfa_l3.l_val)
        b.append(hfa_l3.b)
        fce.append(hfa_l3.fce * 1000.0)
        fuh.append(hfa_l3.fuh * 1000.0)
        fpe.append(hfa_l3.fpe * 1000.0)
        dne.append(hfa_l3.dne)
    
    td = np.array([Time(t) + 30.0 for t in ts])
    store_data('hfa_l3_r', data={'x': td, 'y': r})
    options('hfa_l3_r', 'ytitle', 'R[RE]')
    store_data('hfa_l3_glat', data={'x': td, 'y': glat})
    options('hfa_l3_glat', 'ytitle', 'GMAT[deg]')
    store_data('hfa_l3_mlt', data={'x': td, 'y': mlt})
    options('hfa_l3_mlt', 'ytitle', 'MLT[hour]')
    store_data('hfa_l3_L', data={'x': td, 'y': l_val})
    options('hfa_l3_L', 'ytitle', 'L-val')
    store_data('hfa_l3_b', data={'x': td, 'y': b})
    options('hfa_l3_b', 'ytitle', 'B[nT]')
    store_data('hfa_l3_fce', data={'x': td, 'y': fce})
    options('hfa_l3_fce', 'ytitle', 'Fce[kHz]')
    store_data('hfa_l3_fuh', data={'x': td, 'y': fuh})
    options('hfa_l3_fuh', 'ytitle', 'Fuhr[kHz]')
    store_data('hfa_l3_fpe', data={'x': td, 'y': fpe})
    options('hfa_l3_fpe', 'ytitle', 'Fpe[kHz]')
    store_data('hfa_l3_ne', data={'x': td, 'y': dne})
    options('hfa_l3_ne', 'ytitle', 'Ne[/cc]')
    
    fzc = np.sqrt((fce/2.0)**2 + fpe**2) - fce/2.0  # z-mode cutoff. [kHz]
    store_data('hfa_l3_fzc', data={'x': td, 'y': fzc})
    options('hfa_l3_fzc', 'ytitle', 'Fzc[kHz]')
    
    fpi = fpe * np.sqrt(1.6726219E-27 / (9.1093E-31))
    fcp = fce * 1.6726219E-27 / (9.1093E-31)
    flh = 1.0/np.sqrt(1.0/(fce*fcp) + 1.0/(fpi**2))
    store_data('hfa_l3_flh', data={'x': td, 'y': flh})
    options('hfa_l3_flh', 'ytitle', 'Flh[kHz]')

    return


In [ ]:
; usage: ERG> load_fufp_txt, file, trange=trange, err=err, ver=ver
; ex1)   ERG> time_span, '2017-04-01'
;        ERG> load_fufp_txt
; ex1)   ERG> time_span, '2017-04-01'
;        ERG> load_fufp_txt, ver='v01_03'
; ex1)   ERG> load_fufp_txt, 'Z:\uhrdata\hfa_l3\2017\hfa_l3_20170401_v01_02.txt'
;
;------------------------------------------------------------------
;
pro load_fufp_txt, trange=trange, err=err, file_src=file_src, high=high

  err=0
  if keyword_set(trange) then timespan, trange
;  template_file = 'C:\Data\ERG\HFA\L3\text\hfa_l3_ne_temp_v03.sav'
;  template_file = '/Users/ampuku/Documents/duct/code/IDL/for_event_analysis/uhrHtool/hfa_l3_ne_temp_v03.sav'
  template_file = 'C:\Users\tsuch\IDLWorkspace\ampuku\erg_duct_calculation_IDL\for_event_analysis\uhrHtool\hfa_l3_ne_temp_v03.sav'
  restore, filename = template_file

  ;-------------------------------
  ; Download HFA L3 electron density data
  ;-------------------------------
  if keyword_set(high) then begin
    remotedir = 'http://adrastea.gp.tohoku.ac.jp/~erg/data/hfa_l3_h/'
    localdir = root_data_dir() + 'ERG\HFA\L3\text\'
    relfpathfmt = 'YYYY/erg_hfa_l3_high_YYYYMMDD.txt'
  endif else begin
    remotedir = 'http://adrastea.gp.tohoku.ac.jp/~erg/data/hfa_l3/'
    localdir = root_data_dir() + 'ERG\HFA\L3\text\'
    relfpathfmt = 'YYYY/hfa_l3_YYYYMMDD_v??_??.txt'    
  endelse
  
  relfpaths = file_dailynames(file_format=relfpathfmt, trange=trange, times=times)
  datfiles = $
    spd_download( remote_file = relfpaths, $
    remote_path = remotedir, local_path = localdir, url_username='erg_project', url_password='geospace', /last_version, $
    /force_download, _extra=_extra )

  ; Number of data files
  nd = n_elements(datfiles)

  ;-------------------------------
  ; read HFA L3 electron density data
  ;-------------------------------
  if file_exist(datfiles[0]) ne 1 then begin
    err=1
    return
  endif

  ts    = []
  r     = []
  glat  = []
  mlt   = []
  l_val = []
  b     = []
  fce   = []
  fuh   = []
  fpe   = []
  dne   = []

  file_src = file_basename(datfiles[nd/2])
  for k = 0, nd-1 do begin
    
    hfa_l3 = read_ascii(datfiles[k], template=hfa_l3_ne_temp, count=count)
    if count eq 0 then continue

    n = n_elements(hfa_l3.date)

    for i=0, n-1 do begin
      yr = strmid(hfa_l3.date[i],0,4)
      mn = strmid(hfa_l3.date[i],4,2)
      dy = strmid(hfa_l3.date[i],6,2)
      hr = fix(hfa_l3.hour[i])
      mm = fix(fix(hfa_l3.hour[i]*60)   - hr*60)
      sc = fix(fix(hfa_l3.hour[i]*3600) - hr*3600 - mm*60)
      ts = [ts, string(yr,mn,dy,hr,mm,sc, format='(i4,"-",i2.2,"-",i2.2,"/",i2.2,":",i2.2,":",i2.2)')]
    endfor

    r     = [ r,    hfa_l3.r              ]
    glat  = [ glat, hfa_l3.glat           ]
    mlt   = [ mlt,  hfa_l3.mlt            ]
    l_val = [ l_val,hfa_l3.l_val          ]
    b     = [ b,    hfa_l3.b              ]
    fce   = [ fce,  hfa_l3.fce * 1000.0  ]
    fuh   = [ fuh,  hfa_l3.fuh * 1000.0  ]
    fpe   = [ fpe,  hfa_l3.fpe * 1000.0  ]
    dne   = [ dne,  hfa_l3.dne            ]

  endfor

  
  td = time_double(ts); + 30.0
  store_data, 'hfa_l3_r',    data={x:td,y:r}
  options, 'hfa_l3_r', 'ytitle', 'R[RE]'
  store_data, 'hfa_l3_glat', data={x:td,y:glat}
  options, 'hfa_l3_glat','ytitle', 'GMAT[deg]'
  store_data, 'hfa_l3_mlt',  data={x:td,y:mlt}
  options, 'hfa_l3_mlt','ytitle','MLT[hour]'
  store_data, 'hfa_l3_L',    data={x:td,y:l_val}
  options, 'hfa_l3_L', 'ytitle','L-val'
  store_data, 'hfa_l3_b',    data={x:td,y:b}
  options, 'hfa_l3_b','ytitle','B[nT]'
  store_data, 'hfa_l3_fce',  data={x:td,y:fce}
  options, 'hfa_l3_fce','ytitle','Fce[kHz]'
  store_data, 'hfa_l3_fuh',  data={x:td,y:fuh};, dlim={colors:5,thick:2}
  options, 'hfa_l3_fuh','ytitle','Fuhr[kHz]'
  store_data, 'hfa_l3_fpe',  data={x:td,y:fpe};, dlim={colors:5,thick:2,linestyle:2}
  options, 'hfa_l3_fpe','ytitle','Fpe[kHz]'
  store_data, 'hfa_l3_ne',   data={x:td,y:dne}
  options, 'hfa_l3_ne','ytitle','Ne[/cc]'
  
  fzc = sqrt( (fce/2.0)^2 + (fpe)^2 ) - (fce)/2.0  ; z-mode cutoff. [kHz]
  store_data, 'hfa_l3_fzc',  data={x:td,y:fzc};,               dlim={colors:5,thick:2,linestyle:2}
  options, 'hfa_l3_fzc','ytitle','Fzc[kHz]'

  fpi = fpe * sqrt(1.6726219D * 10^(-27.) / (9.1093D * 10^(-31.)))
  fcp = fce * 1.6726219D * 10^(-27.) / (9.1093D * 10^(-31.))
  flh = 1.0/sqrt(1.0/(fce*fcp)+1.0/(fpi*fpi))
  store_data, 'hfa_l3_flh',  data={x:td,y:flh};,               dlim={colors:4,thick:1}
  options, 'hfa_l3_flh','ytitle','Flh[kHz]'
  
end

In [ ]:
import requests
import os
import numpy as np

def load_fufp_txt(trange=None, high=False):
    err = 0
    if trange is not None:
        timespan(trange)
        
    template_file = r'C:\Users\tsuch\IDLWorkspace\ampuku\erg_duct_calculation_IDL\for_event_analysis\uhrHtool\hfa_l3_ne_temp_v03.sav'
    # 他の変数の定義...
    
    if high:
        remotedir = 'http://adrastea.gp.tohoku.ac.jp/~erg/data/hfa_l3_h/'
        localdir = os.path.join(root_data_dir(), 'ERG', 'HFA', 'L3', 'text')
        relfpathfmt = 'YYYY/erg_hfa_l3_high_YYYYMMDD.txt'
    else:
        remotedir = 'http://adrastea.gp.tohoku.ac.jp/~erg/data/hfa_l3/'
        localdir = os.path.join(root_data_dir(), 'ERG', 'HFA', 'L3', 'text')
        relfpathfmt = 'YYYY/hfa_l3_YYYYMMDD_v??_??.txt'
    
    relfpaths = file_dailynames(file_format=relfpathfmt, trange=trange, times=times)
    # 他の処理...
    
    ts = []
    r = []
    glat = []
    mlt = []
    l_val = []
    b = []
    fce = []
    fuh = []
    fpe = []
    dne = []
    
    file_src = os.path.basename(datfiles[nd//2])
    for k in range(nd):
        hfa_l3 = read_ascii(datfiles[k], template=hfa_l3_ne_temp, count=count)
        if count == 0:
            continue
        
        n = len(hfa_l3.date)
        for i in range(n):
            yr = hfa_l3.date[i][:4]
            mn = hfa_l3.date[i][4:6]
            dy = hfa_l3.date[i][6:8]
            hr = int(hfa_l3.hour[i])
            mm = int(hfa_l3.hour[i]*60) - hr*60
            sc = int(hfa_l3.hour[i]*3600) - hr*3600 - mm*60
            ts.append(f"{yr}-{mn}-{dy}/{hr:02d}:{mm:02d}:{sc:02d}")
        
        r.extend(hfa_l3.r)
        glat.extend(hfa_l3.glat)
        mlt.extend(hfa_l3.mlt)
        l_val.extend(hfa_l3.l_val)
        b.extend(hfa_l3.b)
        fce.extend(hfa_l3.fce * 1000.0)
        fuh.extend(hfa_l3.fuh * 1000.0)
        fpe.extend(hfa_l3.fpe * 1000.0)
        dne.extend(hfa_l3.dne)
    
    td = np.array(ts, dtype='datetime64[s]') + np.timedelta64(30, 's')
    store_data('hfa_l3_r', {'x': td, 'y': r})
    options('hfa_l3_r', 'ytitle', 'R[RE]')
    # 他の処理...


In [2]:
import numpy as np
import urllib.request
from datetime import datetime

def load_fufp_txt(trange=None, high=False):
    err = 0
    if trange is not None:
        timespan(trange)

    #template_file = '/home/jackkooki/Document/duct_analysis/basic/erg_data/satellite/erg/pwe/hfa/event/hfa_l3_ne_temp_v03.sav'
    # restore, filename = template_file  # テンプレートファイルの読み込み（Pythonでは必要ない）

    file_path = '/home/jackkooki/Document/duct_analysis/basic/erg_data/satellite/erg/pwe/hfa/event/hfa_l3_ne_temp_v03.sav'

    with open(file_path, "r") as file:
        contents = file.read()
        print(contents)


    if high:
        remotedir = 'http://adrastea.gp.tohoku.ac.jp/~erg/data/hfa_l3_h/'
    else:
        remotedir = 'http://adrastea.gp.tohoku.ac.jp/~erg/data/hfa_l3/'

    localdir = 'C:/Data/ERG/HFA/L3/text/'
    relfpathfmt = 'YYYY/hfa_l3_YYYYMMDD_v??_??.txt'

    # relfpaths = file_dailynames(file_format=relfpathfmt, trange=trange, times=times)  # ファイルパスの生成（Pythonでは必要ない）
    # datfiles = spd_download(remote_file=relfpaths, remote_path=remotedir, local_path=localdir,
    #                         url_username='erg_project', url_password='geospace', /last_version,
    #                         /force_download, _extra=_extra)  # ファイルのダウンロード（Pythonでは必要ない）

    # Number of data files
    # nd = n_elements(datfiles)  # ファイルの数を取得（Pythonでは必要ない）

    # read HFA L3 electron density data
    # if file_exist(datfiles[0]) ne 1 then begin  # ファイルの存在を確認（Pythonでは必要ない）
    #     err=1
    #     return
    # endif

    ts = []
    r = []
    glat = []
    mlt = []
    l_val = []
    b = []
    fce = []
    fuh = []
    fpe = []
    dne = []

    # file_src = file_basename(datfiles[nd/2])  # ファイル名の取得（Pythonでは必要ない）
    # for k = 0, nd-1 do begin  # ファイルのループ（Pythonでは必要ない）
    #     hfa_l3 = read_ascii(datfiles[k], template=hfa_l3_ne_temp, count=count)  # ファイルの読み込み（Pythonでは必要ない）
    #     if count eq 0 then continue  # countが0の場合、次のファイルに進む（Pythonでは必要ない）

    #     n = n_elements(hfa_l3.date)  # 読み込んだデータの要素数を取得（Pythonでは必要ない）

    #     for i=0, n-1 do begin  # データのループ（Pythonでは必要ない）
    #         yr = strmid(hfa_l3.date[i],0,4)  # 年の取得（Pythonでは必要ない）
    #         mn = strmid(hfa_l3.date[i],4,2)  # 月の取得（Pythonでは必要ない）
    #         dy = strmid(hfa_l3.date[i],6,2)  # 日の取得（Pythonでは必要ない）
    #         hr = fix(hfa_l3.hour[i])  # 時間の取得（Pythonでは必要ない）
    #         mm = fix(fix(hfa_l3.hour[i]*60) - hr*60)  # 分の取得（Pythonでは必要ない）
    #         sc = fix(fix(hfa_l3.hour[i]*3600) - hr*3600 - mm*60)  # 秒の取得（Pythonでは必要ない）
    #         ts = [ts, string(yr,mn,dy,hr,mm,sc, format='(i4,"-",i2.2,"-",i2.2,"/",i2.2,":",i2.2,":",i2.2)')]  # タイムスタンプの生成

    #     r = [ r,    hfa_l3.r              ]  # R[RE]のデータの格納
    #     glat = [ glat, hfa_l3.glat       ]  # GMAT[deg]のデータの格納
    #     mlt = [ mlt,  hfa_l3.mlt         ]  # MLT[hour]のデータの格納
    #     l_val = [ l_val,hfa_l3.l_val     ]  # L-valのデータの格納
    #     b = [ b,    hfa_l3.b             ]  # B[nT]のデータの格納
    #     fce = [ fce,  hfa_l3.fce * 1000.0]  # Fce[kHz]のデータの格納
    #     fuh = [ fuh,  hfa_l3.fuh * 1000.0]  # Fuhr[kHz]のデータの格納
    #     fpe = [ fpe,  hfa_l3.fpe * 1000.0]  # Fpe[kHz]のデータの格納
    #     dne = [ dne,  hfa_l3.dne         ]  # Ne[/cc]のデータの格納

    # endfor

    # td = time_double(ts) + 30.0  # タイムスタンプを数値に変換
    # store_data, 'hfa_l3_r',    data={x:td,y:r}  # データの保存（Pythonでは必要ない）
    # options, 'hfa_l3_r', 'ytitle', 'R[RE]'  # グラフオプションの設定（Pythonでは必要ない）
    # store_data, 'hfa_l3_glat', data={x:td,y:glat}  # データの保存（Pythonでは必要ない）
    # options, 'hfa_l3_glat','ytitle', 'GMAT[deg]'  # グラフオプションの設定（Pythonでは必要ない）
    # store_data, 'hfa_l3_mlt',  data={x:td,y:mlt}  # データの保存（Pythonでは必要ない）
    # options, 'hfa_l3_mlt', 'ytitle', 'MLT[hour]'  # グラフオプションの設定（Pythonでは必要ない）
    # store_data, 'hfa_l3_l',    data={x:td,y:l_val}  # データの保存（Pythonでは必要ない）
    # options, 'hfa_l3_l', 'ytitle', 'L'  # グラフオプションの設定（Pythonでは必要ない）
    # store_data, 'hfa_l3_b',    data={x:td,y:b}  # データの保存（Pythonでは必要ない）
    # options, 'hfa_l3_b', 'ytitle', 'B[nT]'  # グラフオプションの設定（Pythonでは必要ない）
    # store_data, 'hfa_l3_fce',  data={x:td,y:fce}  # データの保存（Pythonでは必要ない）
    # options, 'hfa_l3_fce', 'ytitle', 'Fce[kHz]'  # グラフオプションの設定（Pythonでは必要ない）
    # store_data, 'hfa_l3_fuh',  data={x:td,y:fuh}  # データの保存（Pythonでは必要ない）
    # options, 'hfa_l3_fuh', 'ytitle', 'Fuhr[kHz]'  # グラフオプションの設定（Pythonでは必要ない）
    # store_data, 'hfa_l3_fpe',  data={x:td,y:fpe}  # データの保存（Pythonでは必要ない）
    # options, 'hfa_l3_fpe', 'ytitle', 'Fpe[kHz]'  # グラフオプションの設定（Pythonでは必要ない）
    # store_data, 'hfa_l3_dne',  data={x:td,y:dne}  # データの保存（Pythonでは必要ない）
    # options, 'hfa_l3_dne', 'ytitle', 'Ne[/cc]'  # グラフオプションの設定（Pythonでは必要ない）

    return ts, r, glat, mlt, l_val, b, fce, fuh, fpe, dne


In [6]:
file_path = '/home/jackkooki/Document/duct_analysis/basic/erg_data/satellite/erg/pwe/hfa/event/hfa_l3_ne_temp_v03.sav'

with open(file_path, "rb") as file:
    contents = file.read()
    print(contents)

localdir = 'C:/Data/ERG/HFA/L3/text/'
relfpathfmt = 'YYYY/hfa_l3_YYYYMMDD_v??_??.txt'

# relfpaths = file_dailynames(file_format=relfpathfmt, trange=trange, times=times)  # ファイルパスの生成（Pythonでは必要ない）
# datfiles = spd_download(remote_file=relfpaths, remote_path=remotedir, local_path=localdir,
#                         url_username='erg_project', url_password='geospace', /last_version,
#                         /force_download, _extra=_extra)  # ファイルのダウンロード（Pythonでは必要ない）

# Number of data files
# nd = n_elements(datfiles)  # ファイルの数を取得（Pythonでは必要ない）

# read HFA L3 electron density data
# if file_exist(datfiles[0]) ne 1 then begin  # ファイルの存在を確認（Pythonでは必要ない）
#     err=1
#     return
# endif

ts = []
r = []
glat = []
mlt = []
l_val = []
b = []
fce = []
fuh = []
fpe = []
dne = []


b"SR\x00\x04\x00\x00\x00\n\x00\x00\x04T\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

In [7]:
ts

[]

In [5]:
file_path = '/home/jackkooki/Document/duct_analysis/basic/erg_data/satellite/erg/pwe/hfa/event/hfa_l3_ne_temp_v03.sav'
with open(file_path, "rb") as file:
    contents = file.read()
    print(contents)


b"SR\x00\x04\x00\x00\x00\n\x00\x00\x04T\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\